In [6]:
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=120);

In [3]:
diffs = 0.3, -0.1

In [4]:
assert sum(map(abs,diffs)) <= 1, f'specified gaps are exceed total range: {sum(map(abs,diffs))} > 1'

In [10]:
np.logspace(-1, 2, 5)

array([  0.1       ,   0.56234133,   3.16227766,  17.7827941 ,
       100.        ])

In [16]:
lims = np.zeros([2]*len(diffs))
lims.shape

(2, 2)

In [17]:
all = [slice(None)]*len(diffs)
for i, diff in enumerate(diffs):
	sel = all.copy()
	sel[i] = 1
	lims[tuple(sel)] += diff
lims -= lims.min()

In [21]:
mxlim = lims.max()
# bias = np.random.uniform(0, 1-lims.max())
temp = np.random.uniform(0, 1, lims.shape)

0.5752624887573791

In [23]:
mx_temp = -np.log(mxlim)
mx_temp

0.916290731874155

In [253]:
lims# * np.exp(mx_temp)

array([[0.1, 0. ],
       [0.4, 0.3]])

In [395]:
def sample_numbers(N, gaps):
    # Ensure that sum of gaps is at most 1
    if sum(gaps) > 1:
        raise ValueError("Sum of gaps exceeds 1.")

    # Calculate the remaining space after accounting for the gaps
    remaining_space = 1 - sum(gaps)

    # Uniformly sample N points from the remaining space
    points = np.random.uniform(0, remaining_space, size=N)

    # Sort the points in ascending order
    points.sort()

    # Add each gap to the corresponding point
    for i in range(1, N):
        points[i] += sum(gaps[:i])

    return points


def sample_numbers(N, gaps):
    # Ensure that sum of gaps is at most 1
    if sum(gaps) > 1:
        raise ValueError("Sum of gaps exceeds 1.")

    # Calculate the remaining space after accounting for the gaps
    remaining_space = 1 - sum(gaps)
    # print(remaining_space)

    # Sample arbitrary "spaces" to be added to each gap
    spaces = np.random.uniform(0, 1, size=N-1)
    spaces[0] *= np.sqrt(N-1)
    spaces[-1] *= np.sqrt(N-1)

    # Normalize the spaces so that they sum up to the remaining_space
    spaces = spaces / spaces.sum() * remaining_space


    numbers = spaces.cumsum() + gaps.cumsum()

    # # Generate the numbers by cumulatively adding the gaps and spaces
    # numbers = np.zeros(N)
    # for i in range(1, N):
    #     numbers[i] = numbers[i-1] + gaps[i-1] + spaces[i-1]

    return numbers


In [443]:
gaps = np.asarray([0, 0.1, 0.2, 0.1, 0])  # example gaps
N = len(gaps) + 1
points = sample_numbers(N, gaps)
# print(points)
# print(points[1:] - points[:-1])
probs = points#[:-1]
print(probs)
print(probs[1:] - probs[:-1])

[0.027 0.236 0.564 0.744 1.   ]
[0.209 0.327 0.18  0.256]


In [626]:

def sample_gapped_numbers(gaps, bias=None):
    gaps = np.asarray(gaps)
    squeeze = False
    if gaps.ndim == 1:
        squeeze = True
        gaps = gaps.reshape(1, -1)
    gaps = gaps.T
    D, B = gaps.shape

    if bias is None:
        bias = D

    lims = np.zeros((D+2, B))
    lims[1:-1] = gaps

    coverage = gaps.sum(0,keepdims=True)
    if np.any(coverage > 1):
        raise ValueError("Sum of gaps exceeds 1.")

    remaining = 1 - coverage

    spaces = np.random.uniform(0, 1, size=(D+2, B))
    spaces[0] *= bias
    spaces[-1] *= bias

    spaces = spaces / spaces.sum(0, keepdims=True) * remaining

    numbers = spaces.cumsum(0) + lims.cumsum(0)

    samples = numbers[:-1].T
    if squeeze:
        samples = samples.squeeze()
    return samples

In [875]:
gaps = np.asarray([0.1, 0.2, 0., 0., 0., 0.1])  # example gaps
probs = sample_gapped_numbers(np.tile(gaps, (5,1))).T
print(probs)
print(probs[1:] - probs[:-1])
print((probs[-1]-probs[0]))

[[0.253 0.323 0.105 0.277 0.234]
 [0.382 0.488 0.212 0.415 0.367]
 [0.587 0.723 0.542 0.616 0.616]
 [0.68  0.745 0.562 0.624 0.656]
 [0.687 0.764 0.616 0.641 0.66 ]
 [0.797 0.825 0.698 0.681 0.71 ]
 [0.925 0.932 0.838 0.819 0.857]]
[[0.129 0.165 0.106 0.139 0.133]
 [0.205 0.235 0.33  0.2   0.249]
 [0.093 0.022 0.02  0.008 0.04 ]
 [0.008 0.019 0.054 0.017 0.005]
 [0.11  0.061 0.083 0.04  0.05 ]
 [0.128 0.107 0.14  0.139 0.147]]
[0.672 0.609 0.733 0.543 0.623]


In [886]:
gsel = np.isclose(gaps,0)
gsel

array([False, False,  True,  True,  True, False])

In [887]:
N = 7

sel = np.zeros(N, dtype=bool)
sel[1:] = gsel
sel[:-1] += gsel
sel

array([False, False,  True,  True,  True,  True, False])

In [881]:
probs.T[0]

array([0.253, 0.382, 0.587, 0.68 , 0.687, 0.797, 0.925])

In [890]:
import numpy as np

def find_blobs(arr):
    # Ensure that the array is a numpy array
    if type(arr) != np.ndarray:
        arr = np.array(arr)

    # Add zero at the start and at the end to handle edge blobs
    arr = np.concatenate(([0], arr, [0]))

    # Compute the differences between consecutive elements
    diffs = np.diff(arr)

    # Find the indices where blobs start (1) and end (-1)
    starts = np.where(diffs == 1)[0]
    ends = np.where(diffs == -1)[0]

    # Compute the sizes of the blobs
    sizes = ends - starts

    return starts.tolist(), sizes.tolist()

# arr = np.array(sel)#.astype(bool)
blob_sizes, blob_indices = find_blobs(sel)
print(blob_sizes)
print(blob_indices)

[2]
[4]


In [1]:
import numpy as np

In [2]:

def sample_gapped_numbers(gaps, bias=None):
    gaps = np.asarray(gaps)
    squeeze = False
    if gaps.ndim == 1:
        squeeze = True
        gaps = gaps.reshape(1, -1)
    gaps = gaps.T
    D, B = gaps.shape

    if bias is None:
        bias = D

    lims = np.zeros((D+2, B))
    lims[1:-1] = gaps

    coverage = gaps.sum(0,keepdims=True)
    if np.any(coverage > 1):
        raise ValueError("Sum of gaps exceeds 1.")

    remaining = 1 - coverage

    spaces = np.random.uniform(0, 1, size=(D+2, B))
    spaces[0] *= bias
    spaces[-1] *= bias

    spaces = spaces / spaces.sum(0, keepdims=True) * remaining

    numbers = spaces.cumsum(0) + lims.cumsum(0)

    samples = numbers[:-1].T
    if squeeze:
        samples = samples.squeeze()
    return samples

def find_blobs(arr):
    # Ensure that the array is a numpy array
    if type(arr) != np.ndarray:
        arr = np.array(arr)

    # Add zero at the start and at the end to handle edge blobs
    arr = np.concatenate(([0], arr, [0]))

    # Compute the differences between consecutive elements
    diffs = np.diff(arr)

    # Find the indices where blobs start (1) and end (-1)
    starts = np.where(diffs == 1)[0]
    ends = np.where(diffs == -1)[0]

    # Compute the sizes of the blobs
    sizes = ends - starts

    return starts.tolist(), sizes.tolist()


In [22]:
diffs = 0., 0.

In [23]:
lims = np.zeros([2]*len(diffs))
lims.shape
all = [slice(None)]*len(diffs)
for i, diff in enumerate(diffs):
	sel = all.copy()
	sel[i] = 1
	lims[tuple(sel)] += diff
lims -= lims.min()
lims

array([[0., 0.],
       [0., 0.]])

In [35]:
order = np.argsort(lims.reshape(-1))
line = lims.reshape(-1)[order]
gaps = line[1:] - line[:-1]
samples = sample_gapped_numbers(gaps)
gsel = np.isclose(gaps,0)
sel = np.zeros(len(gaps)+1, dtype=bool)
sel[1:] = gsel
sel[:-1] += gsel
blob_indices, blob_sizes = find_blobs(sel)
for i, n in zip(blob_indices, blob_sizes):
    prev, new = np.arange(n)+i, np.random.permutation(n)+i
    samples[new] = samples[prev]
probs = samples[np.argsort(order)].reshape(lims.shape)
probs

array([[0.379, 0.099],
       [0.556, 0.897]])